# Imports

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

import torch
import rasterio as rio
import numpy as np
import pandas as pd

import utils
import rle2tiff

# HW check

In [ ]:
imgs = utils.get_filenames('input/hm/test', '*.tiff', lambda x: False)
df_train = pd.read_csv('input/hm/HuBMAP-20-dataset_information.csv')

hws = {}
full_names = {}
for f in imgs:
    _, hw, _ = utils.get_basics_rasterio(str(f))
    hws[f.stem] = hw
    full_names[f.stem] = f
hws

In [ ]:
t = df_train[['image_file','width_pixels','height_pixels']]
sanity = {a.split('.')[0]:(c,b) for a,b,c in t.values}
sanity

In [ ]:
for k,v in sanity.items():
    if k in hws:
        if hws[k] != v:
            print('ERR',k,v,hws[k])
        else:
            print('OKK',k,v,hws[k])
            

# Submissions checks

In [ ]:
df = pd.read_csv('output/924/submission_924.csv')
d = df.to_dict()
ids, preds = d['id'], d['predicted']

dd = {}
for k in ids:
    dd[ids[k]] = preds[k]

### Masks from sub

In [ ]:
for k, rle in dd.items():
    hw = hws[k]
    print(k, hw)
    mask  = rle2tiff.rle2mask(rle, hw[::-1])
    mask_name =  f'output/924/predict/masks/{k}_920.tiff'
    utils.save_tiff_uint8_single_band(mask.T, mask_name)
    #merge_name = f'output/924/predict/merged/{k}_924.tiff'
    #utils.tiff_merge_mask(full_names[k], mask_name, merge_name)
    

### DICE between masks

In [ ]:
from loss import dice_loss

In [ ]:
dices = []
for k, rle in dd.items():
    if k != 'aa05346ff': continue
    hw = hws[k]
    print(k, hw)
    mask  = rle2tiff.rle2mask(rle, hw[::-1])
    mymask =  rio.open(f'output/predicts/masks/{k}.tiff').read()
    #mymask =  rio.open(f'output/2021_Mar_17_23_03_21_PAMBUH/masks/{k}.tiff').read()
    
    dice = dice_loss(torch.from_numpy(mask.T/255), torch.from_numpy(mymask))
    print(k, dice)
    dices.append(dice)
    
np.array(dices).mean()

# Merge masks

In [ ]:
for k, rle in dd.items():
    if k != 'aa05346ff': continue
    #if k != '57512b7f1': continue
    #if k != 'd488c759a': continue
    #if k != '2ec3f1bb9': continue
    
    
    hw = hws[k]
    print(k, hw)
    mask_name1 =  f'output/924/predict/masks_924/{k}_924.tiff'
    mask_name2 =  f'output/predicts/masks/{k}.tiff'
    #mask_name2 =  f'output/924/predict/masks/{k}_920.tiff'
    
    merge_name = f'output/924/predict/combined/{k}.tiff'
    
    utils.tiff_merge_mask(full_names[k], mask_name1, merge_name, mask_name2)
    #break